In [1]:
import torch
import numpy as np
import pandas as pd
import torch 

In [2]:
characteristics_pth = '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_0/characteristics'

In [3]:
characteristics = torch.load(characteristics_pth)

In [4]:
characteristics.shape

(2000, 1)

In [5]:
characteristics

array([[ 60.916557],
       [ 39.64073 ],
       [ 25.196112],
       ...,
       [192.10013 ],
       [ 77.49173 ],
       [ 36.524048]], dtype=float32)

In [6]:
import os
import glob
import pdb 
import shutil
import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

# import datetime

# from conf import settings
# import csv
# import argparse

TRTE=False
TRAINERR = False
SELECTED_COL = ['asr', 'auc', 'f1', 'acc','pre','tpr', 'fnr', 'asrd']
ATTACKS_LIST = ['gauss', 'fgsm', 'bim', 'pgd', 'std', 'df', 'cw']

DETECTOR_LIST_LAYERS = ['LID']
DETECTOR_LIST        = ['LID']
CLF = ['LR']


def check_eps(str_val):
    if '8./255.' in str_val:
        eps = '8_255'
    elif '4./255.' in str_val:
        eps = '4_255'     
    elif '2./255.' in str_val:
        eps = '2_255'
    elif '1./255.' in str_val:
        eps = '1_255'
    elif '0.5/255.' in str_val:
        eps = '05_255'
    elif '1./1.' in str_val:
        eps = '1_1'
    elif '1./10.' in str_val:
        eps = '1_10'
    elif '1./100.' in str_val:
        eps = '1_100'
    elif '1./1000.' in str_val:
        eps = '1_1000'

    return eps


def is_float(value):
  try:
    float(value)
    return True
  except:
    return False


def get_clean_accuracy(paths):
    result = {}
    for path in paths:
        if 'gauss' in path:
            attack_method = 'gauss'        
        elif 'fgsm' in path:
            attack_method = 'fgsm'
        elif 'bim' in path:
            attack_method = 'bim'
        elif 'pgd' in path:
            attack_method = 'pgd'
            if 'apgd-ce' in path:
                attack_method = 'apgd-ce'
            elif 'apgd-t' in path:
                attack_method = 'apgd-t'
        elif 'std' in path:
            attack_method = 'std'
            if '8_255' in path:
                eps = '8_255'
            elif '4_255' in path:
                eps = '4_255'     
            elif '2_255' in path:
                eps = '2_255'
            elif '1_255' in path:
                eps = '1_255'
            elif '05_255' in path:
                eps = '05_255'
            elif '1_1' in path:
                eps = '1_1'
            elif '1_10' in path:
                eps = '1_10'
            elif '1_100' in path:
                eps = '1_100'
            elif '1_1000' in path:
                eps = '1_1000'
        elif 'aa+' in path:
            attack_method = 'aa+' 
        elif 'fab-t+' in path:
            attack_method = 'fab-t+'
        elif 'fab+' in path:
            attack_method = 'fab+'
        elif 'square+' in path:
            attack_method = 'square+'
        elif 'fab-t' in path:
            attack_method = 'fab-t'
        elif 'square' in path:
            attack_method = 'square'
        elif 'df' in path:
            attack_method = 'df'
        elif 'cw' in path:
            attack_method = 'cw'
        else:
            raise NotImplementedError("Attack Method not implemented! {}".format(path))
        
        with open(path) as f_attack:
            lines_attack = f_attack.readlines()

        search_text = "INFO: attack success rate:"
        search_text2 = "INFO:  attack success rate:"
        
        if TRTE:
            search_text = "INFO: te attack success rate:"

        asr_list = []
        for line in lines_attack:

            # import pdb; pdb.set_trace()
            if line.__contains__(search_text) or line.__contains__(search_text2):
                asr = float(line.strip().split(' ')[-1])
                asr_list.append(asr)            
                if attack_method  == 'std':
                    result[attack_method + '_' + eps] = asr_list[-1]
                else:
                    result[attack_method] = asr_list[-1]
                    
    return result


def sort_paths_by_layer(paths):
    # './log_evaluation/cif/cif10/run_1/data/detection/cif10/wrn_28_10_10/fgsm/LayerPFS/layer_0/LR/log.txt'
    sorted_paths = sorted(paths, key=lambda x: int(x.split('/')[-2].split('_')[-1]))

    return sorted_paths


In [7]:
OUT_PATH = "analysis_nonlinear_lid/lid_logs"
# OUT_PATH = "analysis/variance/run_gauss_"
APP = '_LAYER'
# APP = '_HPF'
# APP = 'layers'
# APP = ''

LAYERS=True
# LAYERS=False
CSV_FILE_PATH = []
# NR = [1,2,3]
# NR = [8]
# NR = [3]
NR = [1]
    
architecture='wrn_28_10_10'

root='../data'
net=['cif10']; 
dest='../data/extracted_characteristics'; nr=NR; csv_filename='cif10{}.csv'.format(APP); layers=True; ATTACKS='attacks'; 
DETECTION='extracted_characteristics';  k=5
print( ' Extract information! ' )

if not architecture == '':
    architecture = '/' + architecture


# output_path = ''
final = []
paths = [] 
for net_path in net:
    if ATTACKS == 'attacks' and DETECTION == 'detection':
        in_dir_attacks = os.path.join( root, ATTACKS,   'run_' + str(nr), net_path )
        in_dir_detects = os.path.join( root, DETECTION, 'run_' + str(nr), net_path )
    elif  ATTACKS == 'attacks' and DETECTION == 'extracted_characteristics':
        in_dir_attacks = os.path.join( root, ATTACKS,   'run_' + str(nr), net_path )
        in_dir_detects = os.path.join( root, DETECTION, 'run_' + str(nr), net_path )

    # import pdb; pdb.set_trace()

    clean_acc = get_clean_accuracy( glob.glob( in_dir_attacks + architecture + "/**/log.txt", recursive=True ) )
    print("clean accuracy: ", clean_acc)

    if layers:
        detectors = DETECTOR_LIST_LAYERS
    else:
        detectors = DETECTOR_LIST

    # paths = []  
    for det in detectors:
        for classifier in CLF:
            for att in  ATTACKS_LIST:
                lr_paths = []
                if DETECTION == 'extracted_characteristics':
                    if layers:
                        if att == 'std':
                            search_path = in_dir_detects + architecture + "/**/" + att + "/8_255/**/" + det + "/k_{}/layer_*/log.txt".format(k)
                        else:
                            search_path = in_dir_detects + architecture + "/**/" + att + "/" + det + "/k_{}/layer_*/log.txt".format(k)

                        lr_paths = sort_paths_by_layer( glob.glob( search_path, recursive=True) ) 
                    else:
                        if att == 'std':
                            search_path = in_dir_detects + architecture + "/**/" + att + "/8_255/**/" + det + "/" + classifier + "/log.txt"
                        else:
                            search_path = in_dir_detects + architecture + "/**/" + att + "/" + det + "/" + classifier + "/log.txt"

                        lr_paths = glob.glob( search_path, recursive=True)

                elif DETECTION == 'attack_transfer':
                        if att == 'std':
                            search_path = in_dir_detects + architecture + "/**/" + att + "/**/" + det + "/" + classifier + "/**" + "/log.txt"
                        else:
                            search_path = in_dir_detects + architecture + "/**/" + att + "/" + det + "/" + classifier + "/**" + "/log.txt"

                        lr_paths = glob.glob( search_path, recursive=True)

                elif DETECTION == 'data_transfer':
                        if att == 'std':
                            search_path = in_dir_detects + architecture + "/**/" + att + "/**/" + det + "/" + classifier + "/**" + "/log.txt"
                        else:
                            search_path = in_dir_detects + architecture + "/**/" + att + "/" + det + "/" + classifier + "/**" + "/log.txt"

                        lr_paths = glob.glob( search_path, recursive=True)
                else:
                    print("Not known!")

                print("lr_paths: ", lr_paths)

                paths = paths + lr_paths 
print(paths)

 Extract information! 
clean accuracy:  {'apgd-t': 1.0, 'std_4_255': 1.0, 'std_8_255': 1.0, 'std_2_255': 0.9440585653462312, 'std_1_255': 0.5639090796811264, 'std_05_255': 0.23137482391742975, 'pgd': 1.0, 'apgd-ce': 1.0, 'aa+': 1.0, 'fab-t': 1.0, 'bim': 1.0, 'square': 0.98583984375, 'fab-t+': 0.99951171875, 'cw': 1.0, 'fab+': 0.99951171875, 'fgsm': 0.4758522727272727, 'df': 1.0, 'square+': 0.99091796875, 'gauss': 1.0}
lr_paths:  ['../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_0/log.txt', '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_1/log.txt', '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_2/log.txt', '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_3/log.txt', '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_4/log.txt', '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_5/log.txt', '../data/extra

In [8]:
path = paths[0]

In [9]:
path

'../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_0/log.txt'

In [10]:
splitted = path.split('/')
splitted

['..',
 'data',
 'extracted_characteristics',
 'run_1',
 'cif10',
 'wrn_28_10_10',
 'gauss',
 'LID',
 'k_5',
 'layer_0',
 'log.txt']

In [11]:
meta_info = {}
meta_info['path'] =     '/'.join(path.split('/')[:-1])
meta_info['layer'] =    splitted[-2].split('_')[-1]
meta_info['k'] =        splitted[-3].split('_')[-1]
meta_info['attack'] =   splitted[-5]
meta_info['architecture'] = splitted[-6]
meta_info['dataset'] =  splitted[-7]
meta_info['run'] =      splitted[-8].split('_')[-1]

meta_info

{'path': '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_0',
 'layer': '0',
 'k': '5',
 'attack': 'gauss',
 'architecture': 'wrn_28_10_10',
 'dataset': 'cif10',
 'run': '1'}

In [12]:
meta_info['path']

'../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_0'

In [13]:
charact     = torch.load(meta_info['path'] + os.sep + 'characteristics') 
charact_adv = torch.load(meta_info['path'] + os.sep + 'characteristics_adv')

In [14]:
charact[0]

array([60.916557], dtype=float32)

In [15]:
charact_adv[0]

array([45.696968], dtype=float32)

In [16]:
meta_infos = {}

for iter in range(1):
    meta_info = {}
    meta_info['path'] =         '/'.join(path.split('/')[:-1])
    meta_info['layer'] =        splitted[-2].split('_')[-1]
    meta_info['k'] =            splitted[-3].split('_')[-1]
    meta_info['attack'] =       splitted[-5]
    meta_info['architecture'] = splitted[-6]
    meta_info['dataset'] =      splitted[-7]
    meta_info['run'] =          splitted[-8].split('_')[-1]
    
    print( charact[iter])
    meta_info['score'] = charact[iter][0]
    meta_info['score_adv'] = charact_adv[iter][0]
    meta_infos[iter] = meta_info
    

[60.916557]


In [17]:
meta_infos

{0: {'path': '../data/extracted_characteristics/run_1/cif10/wrn_28_10_10/gauss/LID/k_5/layer_0',
  'layer': '0',
  'k': '5',
  'attack': 'gauss',
  'architecture': 'wrn_28_10_10',
  'dataset': 'cif10',
  'run': '1',
  'score': 60.916557,
  'score_adv': 45.696968}}

In [18]:
pd.DataFrame.from_dict(meta_infos, orient='index')

,path,layer,k,attack,architecture,dataset,run,score,score_adv
0,../data/extracted_characteristics/run_1/cif10/...,0,5,gauss,wrn_28_10_10,cif10,1,60.916557,45.696968


In [19]:
out_path = os.path.join('lid_logs', 'k_' + str(k) + '.csv')
# out_path = os.path.join(OUT_PATH, 'k_' + str(k) + '.csv')
out_path

'lid_logs/k_5.csv'

In [20]:
for i in range(1):
    print("i>", i)

i> 0
